In [1]:
pip install torch_geometric

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
import matplotlib.pyplot as plt
from tqdm import tqdm
class Encoder(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(Encoder, self).__init__()
        
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        # self.out = torch.nn.Linear(hidden_channels, 2)
    
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = F.dropout(x, training = self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = global_mean_pool(x, batch)
        # print("x.shape_after_global_mean_pool:", x.shape)
        # x = F.dropout(x, training = self.training)
        # print("x.shape_after_dropout: ", x.shape)
        # x = self.out(x)
        return x

class Decoder(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features):
        super(Decoder, self).__init__()
        
        self.conv1 = GCNConv(hidden_channels, num_node_features)
        self.conv2 = GCNConv(num_node_features, num_node_features)
        self.conv3 = GCNConv(num_node_features, num_node_features)
        
        # self.out = torch.nn.Linear(num_node_features, 2)
    
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = F.dropout(x, training = self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        # x = self.out(x)
        return x

class Autoencoder(torch.nn.Module):  
    def __init__(self, num_node_features, hidden_channels):  
        super().__init__()  
        self.encoder = Encoder(num_node_features, hidden_channels)  
        self.decoder = Decoder(hidden_channels, num_node_features) 
    def forward(self, x, edge_index, batch):  
        # Encode the input  
        
        
        encoded = self.encoder(x, edge_index, batch)
        #added
        # print("encode1:", encoded[0,:10])
        # print("encode2:", encoded[1,:10])
        # print("encoded.unsqueeze(0).shape:", encoded.unsqueeze(0).shape)
        
        encode_copy = encoded.repeat(x.shape[0], 1)
        # print("encode_copy.shape:", encode_copy.shape)
        decoded = self.decoder(encode_copy, edge_index, batch)
        # print("decoded.shape:", decoded.shape)
        
        #added
        def check(features):
            column_mean = features.mean(dim=0, keepdim=True)
            # print("column_mean:", column_mean.shape)
            var = (features - column_mean)**2
            mse = var.mean(dim=0)
            smse = mse.sum()
            return smse
        # print("smse:", check(decoded).item())
        # print("1:", decoded[0, :10])
        # print("2:", decoded[1, :10])
        # print("decode:", decoded) 
        # print("decoded.shape", decoded.shape)
        
        # decoded_of_encode_copy = self.decoder(encode_copy, edge_index, batch)
        
        # print("decoded_of_encode_copy:", decoded_of_encode_copy)
        # print("decoded_of_encode_copy.shape", decoded_of_encode_copy.shape)
        return decoded
    
    
num_nodes = 10
num_node_features = 32
hidden_channels = 512

node_features = torch.randn((num_nodes, num_node_features))
#试一下sigmoid，之后再改回minmax
#好吧事实证明没什么用，甚至loss还更高了点，到了0.04了，minmax还只有0.03几
node_features = torch.sigmoid(node_features)
# minmax=lambda x:(x-x.min())/(x.max()-x.min())
# node_features = minmax(node_features)
edge_index = []
for i in range(num_nodes):
    for j in range(num_nodes):
        if i < j:
            edge_index.append([i,j])
            
            
edge_index = torch.tensor(edge_index).t().contiguous()
data = Data(x = node_features, edge_index = edge_index)


# encode = Encoder(num_node_features = num_node_features, hidden_channels=hidden_channels)#不能直接写数字初始化吗？好像也行
# decode = Decoder(hidden_channels = hidden_channels, num_node_features = num_node_features)
model = Autoencoder(num_node_features = num_node_features, hidden_channels = hidden_channels)
device = torch.device('cpu')
model = model.to(device)
data = data.to(device)
batch = torch.zeros(data.num_nodes, dtype=torch.long).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)#这里parameters可以不加括号？buxing

model.train()


loss_values = []

# 训练过程
for epoch in tqdm(range(10000)):

    optimizer.zero_grad()
    out = model(data.x, data.edge_index, batch)  # 注意这里增加了batch参数
    # print("out1:", data.x[0,:10])
    # print("out2:", data.x[1,:10])
    # 使用全图的标签计算损失
    loss = F.mse_loss(out, data.x)
    # print("loss:",loss.item())
    loss.backward()
    optimizer.step()

    loss_values.append(loss.item())
    if (epoch) % 100 == 0 and epoch >  9000:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


# 绘制损失曲线
plt.figure(figsize=(10, 5))
plt.plot(loss_values, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
# 测试模式
model.eval()






  0%|          | 0/10000 [00:00<?, ?it/s]

 46%|████▌     | 4616/10000 [00:29<00:34, 157.48it/s]

In [3]:
import math
print(math.sqrt(0.03))

0.17320508075688773


**提开个坑在这**（为了之后调原始数据能方便点）

1.调库（运算：torch,torch_geometric,torch.nn.functional,）

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
import tqdm
import random
from matplotlib import pyplot as plt




## 导入原始数据

In [ ]:
import pandas
import torch
import os

def resize_tensor(tensor, target_length):
    tensor = tensor.unsqueeze(0) #增加一个维度，方便resize
    resized_tensor = F.interpolate(
        tensor, 
        size=target_length, 
        mode='linear', 
        align_corners=False
        )
    return resized_tensor.squeeze(0)
all_graphs = [] #所有图数据，一个文件代表一个图



    

# 读取数据
data = pandas.read_csv('/home/featurize/work/ylx/MEA/data')

        